In [57]:
# Importação de bibliotecas relacionadas ao tratamento dos dados
import pandas as pd
import numpy as np
import requests

In [61]:
# Necessária a instalação desta bibliotecas relacionadas a api google maps
# !pip install gmaps
# !pip install -U googlemaps

In [58]:
# Importação de bibliotecas relacionadas ao google maps
from google.colab import output
output.enable_custom_widget_manager()
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries
from googlemaps import Client
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Função de carregamento do dataset
def load_dataset(file):
  path = f'/content/drive/MyDrive/projeto_final_ia/datasets/{file}.csv'
  return pd.read_csv(path)

def save_dataset(dataset, file):
  path = f'/content/drive/MyDrive/projeto_final_ia/datasets/{file}.csv'
  return dataset.to_csv(path, index=False)

In [ ]:
# Carregando dataset
df = load_dataset('joined_dataset')

In [ ]:
# Sequenciando dataset pela feature station_sequence
df.sort_values('station_sequence', ascending=True, inplace=True)

In [ ]:
# Convertendo tipo object de features relacionadas a geolocalização para float
df['station_initial_latitude'] = df['station_initial_latitude'].str.replace(',', '.')
df['station_initial_latitude'] = df['station_initial_latitude'].astype(float)

df['station_initial_longitude'] = df['station_initial_longitude'].str.replace(',', '.')
df['station_initial_longitude'] = df['station_initial_longitude'].astype(float)

In [ ]:
# Geração das features station_final_latitude e station_final_longitude
df['station_final_latitude'] = df['station_initial_latitude'].shift(-1)
df['station_final_longitude'] = df['station_initial_longitude'].shift(-1)

In [62]:
# Integração com api google maps, visando gerar duas novas features: distance e duration
google_api_key = ''

In [ ]:
initial_latitude = df['station_initial_latitude'].tolist()
initial_longitude = df['station_initial_longitude'].tolist()
final_latitude = df['station_final_latitude'].tolist()
final_longitude = df['station_final_longitude'].tolist()

distance = []
duration = []
path_api = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins='

for final_lon in range(len(final_longitude)):
  url = f'{path_api}{initial_latitude[final_lon]},{initial_longitude[final_lon]}&destinations={final_latitude[final_lon]}%2C{final_longitude[final_lon]}&key={google_api_key}'
  response = requests.get(url)
  data = response.json()
  try:
    distance.append(data['rows'][0]['elements'][0]['distance']['text'])
    duration.append(data['rows'][0]['elements'][0]['duration']['text'])
  except KeyError as e:
    print(f"KeyError: Missing key {e} in the API response for index {final_lon}")
  except Exception as e:
    print(f"An error occurred: {e}")

KeyError: Missing key 'distance' in the API response for index 13095


In [ ]:
# Adicionando valor de distance e duration no último registro
distance.append('0 km')
duration.append('0 mins')

In [ ]:
# Removendo string 'm' e 'km' dos valores de distância e convertendo os valores que utilizam unidade de medida km para metros
formated_distance = []
for i in range(len(distance)):
  if distance[i].endswith(' m'):
    modified_item = float(distance[i].rstrip(' m'))
    formated_distance.append(modified_item)
  else:
    modified_item_km = float(distance[i].rstrip(' km')) * 1000
    formated_distance.append(modified_item_km)

df['distance'] = formated_distance

In [ ]:
# Removendo string 'mins' e min dos valores de duration e convertendo valores de unidade de medida minuto para segundos
formated_duration = []
for i in range(len(duration)):
  if(duration[i] != '1 min'):
    formated_duration.append(float((duration[i].replace(' mins', ''))) * 60)
  else:
    formated_duration.append(float((duration[i].replace(' min', ''))) * 60)
df['duration'] = formated_duration

In [ ]:
# Salvando dataset
save_dataset(df,'dataset_integrated_gmaps')

In [ ]:
# Testando carregamento do dataset
df_integrated = load_dataset('dataset_integrated_gmaps')

In [ ]:
df_integrated.head()

,line_id,line_name,line_payment_method,line_category,line_color,station_name,station_code,station_initial_latitude,station_initial_longitude,station_sequence,...,accessibility_info_time,accessibility_info_day,accessibility_info_eletric_ramp,itinerary_company_name,itinerary_code,itinerary_special_line,station_final_latitude,station_final_longitude,distance,duration
0,214,TINGUI,S,ALIMENTADOR,LARANJA,Terminal Cabral - 214 - Tingui,104410,-25.406891,-49.252828,1,...,14:15,1,ELEVADOR,GLÓRIA/PONTUAL,184,N,-25.433741,-49.270292,5100.0,900.0
1,500,LIGEIRÃO BOQUEIRÃO,N,LIGEIRÃO,VERMELHA,Estação Tubo Praça Carlos Gomes,108065,-25.433741,-49.270292,1,...,14:44,1,RAMPA ELÉTRICA,SORRISO/PIONEIRO,1185,N,-25.396167,-49.358341,17300.0,1620.0
2,912,JOSÉ CULPI,S,ALIMENTADOR,LARANJA,"Rua José Culpi, 693 - Butiatuvinha",190324,-25.396167,-49.358341,1,...,05:00,1,ELEVADOR,MERCÊS/PONTUAL,40,N,-25.435187,-49.271813,11300.0,1680.0
3,475,CANAL BELÉM,S,CONVENCIONAL,AMARELA,Praça Rui Barbosa - 475 - Canal Belém,102019,-25.435187,-49.271813,1,...,09:15,3,ELEVADOR,SÃO JOSÉ/PIONEIRO,210,N,-25.436659,-49.273807,1100.0,240.0
4,560,ALFERES POLI,S,CONVENCIONAL,AMARELA,Praça Rui Barbosa - 560 - Alferes Poli,102017,-25.436659,-49.273807,1,...,07:40,2,SEM ELEVADOR,CCD/PIONEIRO,64,N,-25.435052,-49.275876,2000.0,480.0


In [63]:
# Montando uma lista com alguns registros relacionados a geolocalização
features = [
    'station_initial_latitude',
    'station_initial_longitude',
    'station_final_latitude',
    'station_final_longitude'
    ]

features_values = df_integrated[features].iloc[1:10]

list_features_values = features_values.values.tolist()

In [64]:
list_features_values

[[-25.433741254877, -49.270291657395, -25.396166611506, -49.358340564724],
 [-25.396166611506, -49.358340564724, -25.435187289696, -49.271812648587],
 [-25.435187289696, -49.271812648587, -25.43665934794, -49.273806870989],
 [-25.43665934794, -49.273806870989, -25.435052433136, -49.275875855623],
 [-25.435052433136, -49.275875855623, -25.409245409007, -49.36080217424],
 [-25.409245409007, -49.36080217424, -25.41306, -49.20572],
 [-25.41306, -49.20572, -25.401281017147, -49.329734107491],
 [-25.401281017147, -49.329734107491, -25.417850890302, -49.268689037508],
 [-25.417850890302, -49.268689037508, -25.4401504, -49.1830858]]

In [ ]:
# Testando visualização com google maps

# Configurando cliente da API do Google Maps
gmaps.configure(api_key=google_api_key)
gmaps_url = "https://maps.googleapis.com/maps/api/js?key=" + google_api_key
maps = []
for index, item in enumerate(list_features_values):

  # Coordenadas iniciais e finais para o ponto A e o ponto B
  INITIAL_LATITUDE = item[0]
  INITIAL_LONGITUDE = item[1]
  FINAL_LATITUDE = item[2]
  FINAL_LONGITUDE = item[3]

  coordinates_A = (INITIAL_LATITUDE, INITIAL_LONGITUDE)
  coordinates_B = (FINAL_LATITUDE, FINAL_LONGITUDE)

  # Criando mapa centrado entre o ponto A e o ponto B
  fig = gmaps.figure(center=(coordinates_A[0], coordinates_A[1]), zoom_level=16, layout={ 'width': '600px', 'height': '400px' })

  # Adicionando camada de trajetória entre os pontos A e B
  fig.add_layer(gmaps.directions.Directions(coordinates_A, coordinates_B, mode='driving'))

  maps.append(fig)

box_layout = widgets.Layout(border='4px', padding='4px')

# Exibindo mapas das rotas
row1 = widgets.HBox(maps[:5], layout=box_layout)
row2 = widgets.HBox(maps[5:], layout=box_layout)

display(widgets.VBox([row1, row2]))

In [67]:
list_features_values_max_dist = [[-25.358615, -49.241901, -25.596019, -49.274454]]
list_features_values_min_dist = [[-25.435052, -49.275876, -25.436659,	-49.273807]]

In [68]:
# Testando visualização com google maps

# Configurando cliente da API do Google Maps
gmaps.configure(api_key=google_api_key)
gmaps_url = "https://maps.googleapis.com/maps/api/js?key=" + google_api_key
maps = []
for index, item in enumerate(list_features_values_min_dist):

  # Coordenadas iniciais e finais para o ponto A e o ponto B
  INITIAL_LATITUDE = item[0]
  INITIAL_LONGITUDE = item[1]
  FINAL_LATITUDE = item[2]
  FINAL_LONGITUDE = item[3]

  coordinates_A = (INITIAL_LATITUDE, INITIAL_LONGITUDE)
  coordinates_B = (FINAL_LATITUDE, FINAL_LONGITUDE)

  # Criando mapa centrado entre o ponto A e o ponto B
  fig = gmaps.figure(center=(coordinates_A[0], coordinates_A[1]), zoom_level=16, layout={ 'width': '600px', 'height': '400px' })

  # Adicionando camada de trajetória entre os pontos A e B
  fig.add_layer(gmaps.directions.Directions(coordinates_A, coordinates_B, mode='driving'))

  maps.append(fig)

box_layout = widgets.Layout(border='4px', padding='4px')

# Exibindo mapas das rotas
row1 = widgets.HBox(maps[:5], layout=box_layout)
row2 = widgets.HBox(maps[5:], layout=box_layout)

display(widgets.VBox([row1, row2]))

In [69]:
# Testando visualização com google maps

# Configurando cliente da API do Google Maps
gmaps.configure(api_key=google_api_key)
gmaps_url = "https://maps.googleapis.com/maps/api/js?key=" + google_api_key
maps = []
for index, item in enumerate(list_features_values_max_dist):

  # Coordenadas iniciais e finais para o ponto A e o ponto B
  INITIAL_LATITUDE = item[0]
  INITIAL_LONGITUDE = item[1]
  FINAL_LATITUDE = item[2]
  FINAL_LONGITUDE = item[3]

  coordinates_A = (INITIAL_LATITUDE, INITIAL_LONGITUDE)
  coordinates_B = (FINAL_LATITUDE, FINAL_LONGITUDE)

  # Criando mapa centrado entre o ponto A e o ponto B
  fig = gmaps.figure(center=(coordinates_A[0], coordinates_A[1]), zoom_level=16, layout={ 'width': '600px', 'height': '400px' })

  # Adicionando camada de trajetória entre os pontos A e B
  fig.add_layer(gmaps.directions.Directions(coordinates_A, coordinates_B, mode='driving'))

  maps.append(fig)

box_layout = widgets.Layout(border='4px', padding='4px')

# Exibindo mapas das rotas
row1 = widgets.HBox(maps[:5], layout=box_layout)
row2 = widgets.HBox(maps[5:], layout=box_layout)

display(widgets.VBox([row1, row2]))